# Crypto HFT Trading Strategy - Feature Engineering

This notebook implements advanced feature engineering techniques for high-frequency trading of ETH_EUR and XBT_EUR, inspired by the SGX methodology.

## Objectives:
1. Extract microstructure features from order book data
2. Create cross-asset features for lead-lag modeling
3. Generate time-based and technical indicators
4. Implement feature selection and dimensionality reduction
5. Prepare features for machine learning models

In [ ]:
# Import necessary libraries
import sys
import os
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn for feature selection and preprocessing
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

# Import project modules
from data_processing.feature_extractor import OrderBookFeatureExtractor
from data_processing.synchronizer import DataSynchronizer
from utils.visualization import OrderBookVisualizer

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Synchronized Data

In [ ]:
# Load the synchronized data from previous notebook
print("Loading synchronized order book data...")
try:
    sync_data = pd.read_parquet('../data/processed/synchronized_data.parquet')
    print(f"Loaded synchronized data: {sync_data.shape}")
except FileNotFoundError:
    print("Synchronized data not found. Please run 01_data_exploration.ipynb first.")
    # Alternative: load and synchronize data here
    from data_processing.data_loader import OrderBookDataLoader
    
    data_loader = OrderBookDataLoader()
    synchronizer = DataSynchronizer()
    
    eth_data = data_loader.load_order_book_data('../data/parquet/DATA0_ETH_EUR.parquet')
    xbt_data = data_loader.load_order_book_data('../data/parquet/DATA0_XBT_EUR.parquet')
    
    sync_data = synchronizer.synchronize_data(
        {'ETH': eth_data, 'XBT': xbt_data},
        method='linear_interpolation',
        max_gap_ms=1000
    )
    print(f"Created synchronized data: {sync_data.shape}")

print(f"Data timeframe: {sync_data.index.min()} to {sync_data.index.max()}")
print(f"Columns: {list(sync_data.columns)}")

## 2. Basic Microstructure Features

In [ ]:
# Initialize feature extractor
feature_extractor = OrderBookFeatureExtractor()

# Extract features for each asset
print("Extracting ETH microstructure features...")
eth_features = feature_extractor.extract_features(
    sync_data.filter(regex='^ETH_').rename(columns=lambda x: x.replace('ETH_', ''))
)

print("Extracting XBT microstructure features...")
xbt_features = feature_extractor.extract_features(
    sync_data.filter(regex='^XBT_').rename(columns=lambda x: x.replace('XBT_', ''))
)

# Add prefix to distinguish assets
eth_features = eth_features.add_prefix('ETH_')
xbt_features = xbt_features.add_prefix('XBT_')

print(f"ETH features shape: {eth_features.shape}")
print(f"XBT features shape: {xbt_features.shape}")
print(f"\nETH feature columns: {list(eth_features.columns)}")

## 3. Advanced Technical Indicators

In [ ]:
def calculate_advanced_features(data, prefix='', windows=[5, 10, 20, 50]):
    """
    Calculate advanced technical indicators and microstructure features
    """
    features = pd.DataFrame(index=data.index)
    
    # Basic price features
    mid_price = (data['ask_price_1'] + data['bid_price_1']) / 2
    spread = data['ask_price_1'] - data['bid_price_1']
    
    # Price-based features
    features[f'{prefix}_mid_price'] = mid_price
    features[f'{prefix}_log_mid_price'] = np.log(mid_price)
    features[f'{prefix}_spread'] = spread
    features[f'{prefix}_relative_spread'] = spread / mid_price
    
    # Returns at different horizons
    for window in [1, 2, 5, 10]:
        features[f'{prefix}_return_{window}'] = mid_price.pct_change(window)
        features[f'{prefix}_log_return_{window}'] = np.log(mid_price / mid_price.shift(window))
    
    # Volume features
    total_volume = data['bid_quantity_1'] + data['ask_quantity_1']
    volume_imbalance = (data['bid_quantity_1'] - data['ask_quantity_1']) / total_volume
    
    features[f'{prefix}_volume_imbalance'] = volume_imbalance
    features[f'{prefix}_total_volume'] = total_volume
    features[f'{prefix}_log_volume'] = np.log(total_volume + 1)
    
    # Weighted mid price (volume-weighted)
    features[f'{prefix}_weighted_mid'] = (data['bid_price_1'] * data['ask_quantity_1'] + 
                                        data['ask_price_1'] * data['bid_quantity_1']) / total_volume
    
    # Rolling statistics
    for window in windows:
        # Price momentum and mean reversion
        features[f'{prefix}_price_momentum_{window}'] = mid_price.diff().rolling(window).mean()
        features[f'{prefix}_price_mean_reversion_{window}'] = mid_price - mid_price.rolling(window).mean()
        
        # Volatility features
        returns = mid_price.pct_change()
        features[f'{prefix}_volatility_{window}'] = returns.rolling(window).std()
        features[f'{prefix}_realized_vol_{window}'] = (returns ** 2).rolling(window).sum()
        
        # Volume features
        features[f'{prefix}_volume_momentum_{window}'] = total_volume.diff().rolling(window).mean()
        features[f'{prefix}_volume_volatility_{window}'] = total_volume.rolling(window).std()
        
        # Spread features
        features[f'{prefix}_spread_momentum_{window}'] = spread.diff().rolling(window).mean()
        features[f'{prefix}_spread_volatility_{window}'] = spread.rolling(window).std()
        
        # Imbalance features
        features[f'{prefix}_imbalance_momentum_{window}'] = volume_imbalance.diff().rolling(window).mean()
        features[f'{prefix}_imbalance_volatility_{window}'] = volume_imbalance.rolling(window).std()
    
    # Technical indicators
    # RSI (Relative Strength Index)
    delta = mid_price.diff()
    gain = (delta.where(delta > 0, 0)).rolling(14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
    rs = gain / loss
    features[f'{prefix}_rsi'] = 100 - (100 / (1 + rs))
    
    # Bollinger Bands
    bb_window = 20
    bb_mean = mid_price.rolling(bb_window).mean()
    bb_std = mid_price.rolling(bb_window).std()
    features[f'{prefix}_bb_upper'] = bb_mean + 2 * bb_std
    features[f'{prefix}_bb_lower'] = bb_mean - 2 * bb_std
    features[f'{prefix}_bb_position'] = (mid_price - bb_lower) / (bb_upper - bb_lower)
    
    # MACD
    ema_12 = mid_price.ewm(span=12).mean()
    ema_26 = mid_price.ewm(span=26).mean()
    features[f'{prefix}_macd'] = ema_12 - ema_26
    features[f'{prefix}_macd_signal'] = features[f'{prefix}_macd'].ewm(span=9).mean()
    features[f'{prefix}_macd_histogram'] = features[f'{prefix}_macd'] - features[f'{prefix}_macd_signal']
    
    return features

# Calculate advanced features for both assets
print("Calculating advanced ETH features...")
eth_advanced = calculate_advanced_features(
    sync_data.filter(regex='^ETH_').rename(columns=lambda x: x.replace('ETH_', '')),
    prefix='ETH'
)

print("Calculating advanced XBT features...")
xbt_advanced = calculate_advanced_features(
    sync_data.filter(regex='^XBT_').rename(columns=lambda x: x.replace('XBT_', '')),
    prefix='XBT'
)

print(f"ETH advanced features: {eth_advanced.shape}")
print(f"XBT advanced features: {xbt_advanced.shape}")

## 4. Cross-Asset Features and Lead-Lag Relationships

In [ ]:
def calculate_cross_asset_features(eth_data, xbt_data, lags=[1, 2, 3, 5, 10]):
    """
    Calculate cross-asset features to capture lead-lag relationships
    """
    features = pd.DataFrame(index=eth_data.index)
    
    # Price correlation features
    eth_mid = eth_data['ETH_mid_price']
    xbt_mid = xbt_data['XBT_mid_price']
    
    eth_returns = eth_mid.pct_change()
    xbt_returns = xbt_mid.pct_change()
    
    # Rolling correlations
    for window in [10, 20, 50]:
        features[f'price_correlation_{window}'] = eth_returns.rolling(window).corr(xbt_returns)
        features[f'volume_correlation_{window}'] = eth_data['ETH_total_volume'].rolling(window).corr(xbt_data['XBT_total_volume'])
    
    # Lead-lag features
    for lag in lags:
        # ETH leading XBT
        features[f'eth_lead_xbt_return_{lag}'] = eth_returns.shift(lag).rolling(5).corr(xbt_returns)
        features[f'eth_lead_xbt_volume_{lag}'] = eth_data['ETH_total_volume'].shift(lag).rolling(5).corr(xbt_data['XBT_total_volume'])
        
        # XBT leading ETH
        features[f'xbt_lead_eth_return_{lag}'] = xbt_returns.shift(lag).rolling(5).corr(eth_returns)
        features[f'xbt_lead_eth_volume_{lag}'] = xbt_data['XBT_total_volume'].shift(lag).rolling(5).corr(eth_data['ETH_total_volume'])
        
        # Lagged cross-returns
        features[f'eth_return_vs_xbt_lag_{lag}'] = eth_returns * xbt_returns.shift(lag)
        features[f'xbt_return_vs_eth_lag_{lag}'] = xbt_returns * eth_returns.shift(lag)
    
    # Price ratio and spread features
    price_ratio = eth_mid / xbt_mid
    features['price_ratio'] = price_ratio
    features['log_price_ratio'] = np.log(price_ratio)
    features['price_ratio_change'] = price_ratio.pct_change()
    
    # Z-score of price ratio
    for window in [20, 50, 100]:
        ratio_mean = price_ratio.rolling(window).mean()
        ratio_std = price_ratio.rolling(window).std()
        features[f'price_ratio_zscore_{window}'] = (price_ratio - ratio_mean) / ratio_std
    
    # Volume ratio features
    volume_ratio = eth_data['ETH_total_volume'] / xbt_data['XBT_total_volume']
    features['volume_ratio'] = volume_ratio
    features['volume_ratio_change'] = volume_ratio.pct_change()
    
    # Spread differential
    eth_rel_spread = eth_data['ETH_relative_spread']
    xbt_rel_spread = xbt_data['XBT_relative_spread']
    features['spread_differential'] = eth_rel_spread - xbt_rel_spread
    features['spread_ratio'] = eth_rel_spread / xbt_rel_spread
    
    # Imbalance differential
    features['imbalance_differential'] = eth_data['ETH_volume_imbalance'] - xbt_data['XBT_volume_imbalance']
    
    return features

# Calculate cross-asset features
print("Calculating cross-asset features...")
cross_asset_features = calculate_cross_asset_features(eth_advanced, xbt_advanced)

print(f"Cross-asset features shape: {cross_asset_features.shape}")
print(f"Sample cross-asset features: {list(cross_asset_features.columns[:10])}")

## 5. Time-Based Features

In [ ]:
def calculate_time_features(data):
    """
    Calculate time-based features for market microstructure
    """
    features = pd.DataFrame(index=data.index)
    
    # Extract time components
    features['hour'] = data.index.hour
    features['minute'] = data.index.minute
    features['second'] = data.index.second
    features['microsecond'] = data.index.microsecond
    
    # Day of week (0=Monday, 6=Sunday)
    features['day_of_week'] = data.index.dayofweek
    
    # Time since market open (assuming 24/7 crypto markets)
    features['seconds_since_midnight'] = (data.index.hour * 3600 + 
                                        data.index.minute * 60 + 
                                        data.index.second)
    
    # Cyclical encoding for time features
    features['hour_sin'] = np.sin(2 * np.pi * features['hour'] / 24)
    features['hour_cos'] = np.cos(2 * np.pi * features['hour'] / 24)
    features['minute_sin'] = np.sin(2 * np.pi * features['minute'] / 60)
    features['minute_cos'] = np.cos(2 * np.pi * features['minute'] / 60)
    features['dow_sin'] = np.sin(2 * np.pi * features['day_of_week'] / 7)
    features['dow_cos'] = np.cos(2 * np.pi * features['day_of_week'] / 7)
    
    # Market session indicators (assuming different activity patterns)
    features['is_us_session'] = ((features['hour'] >= 14) & (features['hour'] < 22)).astype(int)
    features['is_asia_session'] = ((features['hour'] >= 0) & (features['hour'] < 8)).astype(int)
    features['is_europe_session'] = ((features['hour'] >= 8) & (features['hour'] < 16)).astype(int)
    
    # Weekend indicator
    features['is_weekend'] = (features['day_of_week'] >= 5).astype(int)
    
    return features

# Calculate time features
print("Calculating time-based features...")
time_features = calculate_time_features(sync_data)

print(f"Time features shape: {time_features.shape}")
print(f"Time features: {list(time_features.columns)}")

## 6. Feature Combination and Preprocessing

In [ ]:
# Combine all features
print("Combining all features...")
all_features = pd.concat([
    eth_features,
    xbt_features,
    eth_advanced,
    xbt_advanced,
    cross_asset_features,
    time_features
], axis=1)

print(f"Combined features shape: {all_features.shape}")
print(f"Total number of features: {len(all_features.columns)}")

# Remove infinite and NaN values
print("\nCleaning features...")
print(f"NaN values before cleaning: {all_features.isnull().sum().sum()}")
print(f"Infinite values before cleaning: {np.isinf(all_features.select_dtypes(include=[np.number])).sum().sum()}")

# Replace infinite values with NaN
all_features = all_features.replace([np.inf, -np.inf], np.nan)

# Drop columns with too many NaN values (>50%)
nan_threshold = 0.5
nan_ratio = all_features.isnull().sum() / len(all_features)
columns_to_drop = nan_ratio[nan_ratio > nan_threshold].index
print(f"Dropping {len(columns_to_drop)} columns with >50% NaN values")
all_features = all_features.drop(columns=columns_to_drop)

# Forward fill remaining NaN values (limited to 5 periods)
all_features = all_features.fillna(method='ffill', limit=5)

# Drop remaining rows with NaN values
all_features = all_features.dropna()

print(f"\nFinal features shape after cleaning: {all_features.shape}")
print(f"Remaining NaN values: {all_features.isnull().sum().sum()}")

## 7. Feature Analysis and Selection

In [ ]:
# Feature correlation analysis
print("Analyzing feature correlations...")
correlation_matrix = all_features.corr()

# Find highly correlated features
high_corr_threshold = 0.95
upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

high_corr_pairs = [
    (column, row, upper_triangle.loc[row, column])
    for column in upper_triangle.columns
    for row in upper_triangle.index
    if abs(upper_triangle.loc[row, column]) > high_corr_threshold
]

print(f"Found {len(high_corr_pairs)} highly correlated feature pairs (|corr| > {high_corr_threshold})")

# Remove highly correlated features
features_to_remove = set()
for col1, col2, corr in high_corr_pairs:
    # Keep the feature with more variation (higher std)
    if all_features[col1].std() < all_features[col2].std():
        features_to_remove.add(col1)
    else:
        features_to_remove.add(col2)

print(f"Removing {len(features_to_remove)} highly correlated features")
all_features_filtered = all_features.drop(columns=list(features_to_remove))

print(f"Features after correlation filtering: {all_features_filtered.shape}")

In [ ]:
# Feature importance visualization
# Calculate feature statistics
feature_stats = pd.DataFrame({
    'mean': all_features_filtered.mean(),
    'std': all_features_filtered.std(),
    'skew': all_features_filtered.skew(),
    'kurtosis': all_features_filtered.kurtosis(),
    'min': all_features_filtered.min(),
    'max': all_features_filtered.max()
})

# Plot feature distribution statistics
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Standard deviation distribution
axes[0, 0].hist(feature_stats['std'], bins=50, alpha=0.7)
axes[0, 0].set_title('Distribution of Feature Standard Deviations')
axes[0, 0].set_xlabel('Standard Deviation')
axes[0, 0].set_ylabel('Frequency')

# Skewness distribution
axes[0, 1].hist(feature_stats['skew'], bins=50, alpha=0.7)
axes[0, 1].set_title('Distribution of Feature Skewness')
axes[0, 1].set_xlabel('Skewness')
axes[0, 1].set_ylabel('Frequency')

# Kurtosis distribution
axes[1, 0].hist(feature_stats['kurtosis'], bins=50, alpha=0.7)
axes[1, 0].set_title('Distribution of Feature Kurtosis')
axes[1, 0].set_xlabel('Kurtosis')
axes[1, 0].set_ylabel('Frequency')

# Feature range (max - min)
feature_range = feature_stats['max'] - feature_stats['min']
axes[1, 1].hist(feature_range, bins=50, alpha=0.7)
axes[1, 1].set_title('Distribution of Feature Ranges')
axes[1, 1].set_xlabel('Range (Max - Min)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\nFeature Statistics Summary:")
print(feature_stats.describe())

## 8. Target Variable Creation

In [ ]:
def create_target_variables(data, horizons=[1, 2, 5, 10]):
    """
    Create target variables for prediction at different time horizons
    """
    targets = pd.DataFrame(index=data.index)
    
    # Calculate mid prices
    eth_mid = (data['ETH_ask_price_1'] + data['ETH_bid_price_1']) / 2
    xbt_mid = (data['XBT_ask_price_1'] + data['XBT_bid_price_1']) / 2
    
    for horizon in horizons:
        # Future returns (regression targets)
        targets[f'ETH_future_return_{horizon}'] = eth_mid.pct_change(horizon).shift(-horizon)
        targets[f'XBT_future_return_{horizon}'] = xbt_mid.pct_change(horizon).shift(-horizon)
        
        # Future price direction (classification targets)
        eth_future_return = targets[f'ETH_future_return_{horizon}']
        xbt_future_return = targets[f'XBT_future_return_{horizon}']
        
        # Binary classification: up (1) or down (0)
        targets[f'ETH_direction_{horizon}'] = (eth_future_return > 0).astype(int)
        targets[f'XBT_direction_{horizon}'] = (xbt_future_return > 0).astype(int)
        
        # Ternary classification: up (2), neutral (1), down (0)
        threshold = 0.0001  # 1 basis point
        targets[f'ETH_direction_3class_{horizon}'] = pd.cut(
            eth_future_return,
            bins=[-np.inf, -threshold, threshold, np.inf],
            labels=[0, 1, 2]
        ).astype(float)
        
        targets[f'XBT_direction_3class_{horizon}'] = pd.cut(
            xbt_future_return,
            bins=[-np.inf, -threshold, threshold, np.inf],
            labels=[0, 1, 2]
        ).astype(float)
    
    # Cross-asset momentum targets
    for horizon in horizons:
        eth_ret = targets[f'ETH_future_return_{horizon}']
        xbt_ret = targets[f'XBT_future_return_{horizon}']
        
        # Which asset will outperform
        targets[f'ETH_outperforms_{horizon}'] = (eth_ret > xbt_ret).astype(int)
        
        # Relative return
        targets[f'relative_return_{horizon}'] = eth_ret - xbt_ret
    
    return targets

# Create target variables
print("Creating target variables...")
targets = create_target_variables(sync_data)

print(f"Target variables shape: {targets.shape}")
print(f"Target columns: {list(targets.columns)}")

# Check target distribution
print("\nTarget variable distributions:")
for col in targets.columns:
    if 'direction' in col and '3class' not in col:
        print(f"{col}: {targets[col].value_counts().to_dict()}")

## 9. Feature Scaling and Normalization

In [ ]:
# Prepare final dataset
print("Preparing final feature dataset...")

# Align features and targets
common_index = all_features_filtered.index.intersection(targets.index)
final_features = all_features_filtered.loc[common_index]
final_targets = targets.loc[common_index]

# Remove rows with NaN targets
valid_rows = ~final_targets.isnull().any(axis=1)
final_features = final_features[valid_rows]
final_targets = final_targets[valid_rows]

print(f"Final dataset shape: Features {final_features.shape}, Targets {final_targets.shape}")

# Split into train/validation/test sets
# Use time-based split for financial data
n_samples = len(final_features)
train_size = int(0.6 * n_samples)
val_size = int(0.2 * n_samples)

X_train = final_features.iloc[:train_size]
X_val = final_features.iloc[train_size:train_size + val_size]
X_test = final_features.iloc[train_size + val_size:]

y_train = final_targets.iloc[:train_size]
y_val = final_targets.iloc[train_size:train_size + val_size]
y_test = final_targets.iloc[train_size + val_size:]

print(f"\nData splits:")
print(f"Train: {X_train.shape[0]} samples ({X_train.index.min()} to {X_train.index.max()})")
print(f"Validation: {X_val.shape[0]} samples ({X_val.index.min()} to {X_val.index.max()})")
print(f"Test: {X_test.shape[0]} samples ({X_test.index.min()} to {X_test.index.max()})")

In [ ]:
# Feature scaling
print("Scaling features...")

# Use RobustScaler to handle outliers
scaler = RobustScaler()

# Fit scaler on training data only
X_train_scaled = pd.DataFrame(
    scaler.fit_transform(X_train),
    index=X_train.index,
    columns=X_train.columns
)

X_val_scaled = pd.DataFrame(
    scaler.transform(X_val),
    index=X_val.index,
    columns=X_val.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test),
    index=X_test.index,
    columns=X_test.columns
)

print("Feature scaling completed.")

# Check scaling results
print(f"\nScaled feature statistics (training set):")
print(f"Mean: {X_train_scaled.mean().mean():.6f}")
print(f"Std: {X_train_scaled.std().mean():.6f}")
print(f"Min: {X_train_scaled.min().min():.6f}")
print(f"Max: {X_train_scaled.max().max():.6f}")

## 10. Feature Selection

In [ ]:
# Feature selection using different methods
print("Performing feature selection...")

# Example target for feature selection (ETH 5-step return)
target_col = 'ETH_future_return_5'
y_target = y_train[target_col].dropna()
X_target = X_train_scaled.loc[y_target.index]

# Statistical feature selection (F-test)
k_best_features = min(100, X_target.shape[1] // 2)  # Select top 100 or half of features
selector_f = SelectKBest(score_func=f_regression, k=k_best_features)
X_selected_f = selector_f.fit_transform(X_target, y_target)
selected_features_f = X_target.columns[selector_f.get_support()]

print(f"F-test selected {len(selected_features_f)} features")

# Mutual information feature selection
selector_mi = SelectKBest(score_func=mutual_info_regression, k=k_best_features)
X_selected_mi = selector_mi.fit_transform(X_target, y_target)
selected_features_mi = X_target.columns[selector_mi.get_support()]

print(f"Mutual information selected {len(selected_features_mi)} features")

# Combine both selections
selected_features_combined = list(set(selected_features_f) | set(selected_features_mi))
print(f"Combined selection: {len(selected_features_combined)} features")

# Feature importance scores
feature_scores = pd.DataFrame({
    'feature': X_target.columns,
    'f_score': selector_f.scores_,
    'mi_score': selector_mi.scores_
})

# Plot top features
top_f_features = feature_scores.nlargest(20, 'f_score')
top_mi_features = feature_scores.nlargest(20, 'mi_score')

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# F-test scores
axes[0].barh(range(len(top_f_features)), top_f_features['f_score'])
axes[0].set_yticks(range(len(top_f_features)))
axes[0].set_yticklabels(top_f_features['feature'], fontsize=8)
axes[0].set_xlabel('F-Score')
axes[0].set_title('Top 20 Features by F-Test Score')
axes[0].invert_yaxis()

# Mutual information scores
axes[1].barh(range(len(top_mi_features)), top_mi_features['mi_score'])
axes[1].set_yticks(range(len(top_mi_features)))
axes[1].set_yticklabels(top_mi_features['feature'], fontsize=8)
axes[1].set_xlabel('Mutual Information Score')
axes[1].set_title('Top 20 Features by Mutual Information')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

## 11. Save Processed Features

In [ ]:
# Save processed features and targets with error handling
import os
os.makedirs('../data/processed', exist_ok=True)
try:
    X_train_scaled.to_parquet('../data/processed/X_train_scaled.parquet')
    X_val_scaled.to_parquet('../data/processed/X_val_scaled.parquet')
    X_test_scaled.to_parquet('../data/processed/X_test_scaled.parquet')
    y_train.to_parquet('../data/processed/y_train.parquet')
    y_val.to_parquet('../data/processed/y_val.parquet')
    y_test.to_parquet('../data/processed/y_test.parquet')
    final_features.to_parquet('../data/processed/features_unscaled.parquet')
    final_targets.to_parquet('../data/processed/targets.parquet')
    with open('../data/processed/feature_metadata.json', 'w') as f:
        json.dump(feature_metadata, f, indent=2, default=str)
    import joblib
    joblib.dump(scaler, '../data/processed/feature_scaler.pkl')
    print("Feature engineering completed successfully! Files saved to ../data/processed/")
except Exception as e:
    print(f"Error saving processed features or targets: {e}")

## 12. Feature Engineering Summary

In [ ]:
print("FEATURE ENGINEERING SUMMARY")
print("="*60)

print(f"\n1. FEATURE CATEGORIES CREATED:")
feature_categories = {
    'Basic Microstructure': ['spread', 'volume_imbalance', 'mid_price'],
    'Technical Indicators': ['rsi', 'macd', 'bollinger', 'momentum'],
    'Cross-Asset': ['correlation', 'lead_lag', 'ratio', 'differential'],
    'Time-Based': ['hour', 'minute', 'session', 'cyclical'],
    'Rolling Statistics': ['volatility', 'momentum', 'mean_reversion']
}

for category, keywords in feature_categories.items():
    count = sum(1 for col in final_features.columns 
               if any(keyword in col.lower() for keyword in keywords))
    print(f"   {category}: ~{count} features")

print(f"\n2. DATA QUALITY METRICS:")
print(f"   Original features: {len(all_features.columns)}")
print(f"   After correlation filtering: {len(all_features_filtered.columns)}")
print(f"   Final features: {len(final_features.columns)}")
print(f"   Selected features (F-test): {len(selected_features_f)}")
print(f"   Selected features (MI): {len(selected_features_mi)}")
print(f"   Combined selection: {len(selected_features_combined)}")

print(f"\n3. TARGET VARIABLES:")
print(f"   Return prediction horizons: [1, 2, 5, 10] steps")
print(f"   Direction classification: Binary and ternary")
print(f"   Cross-asset targets: Relative performance")
print(f"   Total target variables: {len(final_targets.columns)}")

print(f"\n4. DATASET SPLITS:")
print(f"   Training: {len(X_train):,} samples (60%)")
print(f"   Validation: {len(X_val):,} samples (20%)")
print(f"   Test: {len(X_test):,} samples (20%)")

print(f"\n5. NEXT STEPS:")
print(f"   ✓ Features ready for ML model training")
print(f"   ✓ Multiple target variables for different strategies")
print(f"   ✓ Time-series split preserves temporal order")
print(f"   ✓ Feature selection identifies most predictive variables")
print(f"   → Ready for model development notebook")

print(f"\n" + "="*60)
print(f"FEATURE ENGINEERING COMPLETED SUCCESSFULLY!")
print(f"All processed data saved to ../data/processed/")

In [ ]:
# Set global seed for reproducibility
import random
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
try:
    import os
    os.environ["PYTHONHASHSEED"] = str(SEED)
except Exception:
    pass

In [ ]:
# Assert required columns exist before feature extraction
required_columns = [
    'bid_price_1', 'ask_price_1', 'bid_quantity_1', 'ask_quantity_1'
]
for col in required_columns:
    assert any(col in c for c in sync_data.columns), f"Missing column {col} in synchronized data"

In [ ]:
# Robust ratio calculation for advanced features (avoid division by zero)
def safe_divide(numerator, denominator):
    return np.where(denominator != 0, numerator / denominator, 0)

# Example usage in feature engineering (replace direct division by safe_divide)
# ...inside calculate_advanced_features and calculate_cross_asset_features, replace all x/y by safe_divide(x, y)

In [ ]:
# Simple output validation (unit-test style)
assert all_features.shape[0] > 0, "No features generated!"
assert final_features.shape[0] > 0, "No final features after cleaning!"
assert final_targets.shape[0] > 0, "No targets generated!"
print("[TESTS PASSED] Key outputs are valid.")